<a href="https://colab.research.google.com/github/ML-Bioinfo-CEITEC/genomic_benchmarks/blob/main/notebooks/How_To_Train_CNN_Classifier_With_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How To Train CNN Classifier With TF

This notebook demonstrates how to use `genomic_benchmarks` to train a neural network classifier on one of its benchmark datasets [human_nontata_promoters](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks/tree/main/docs/human_nontata_promoters).

In [1]:
#If you work in Google Colaboratory - uncomment the following line to install the package to your virtual machine
#!pip install -qq tensorflow_addons genomic-benchmarks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.9 MB/s eta 0:00:00


# Data download

With the function `download_dataset` downloads, we can download full-sequence form of the benchmark, splitted into train and test sets, one folder for each class.

In [2]:
from pathlib import Path
import tensorflow as tf
import numpy as np

from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import get_basic_cnn_model_v0 as get_model

if not is_downloaded('human_nontata_promoters'):
    download_dataset('human_nontata_promoters')

/usr/local/lib/python3.10/dist-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")
Downloa

In [3]:
info('human_nontata_promoters', 0)

Dataset `human_nontata_promoters` has 2 classes: negative, positive.

All lengths of genomic intervals equals 251.

Totally 36131 sequences have been found, 27097 for training and 9034 for testing.


,train,test
negative,12355,4119
positive,14742,4915


## TF Dataset object

To train the model with TensorFlow, we must create a TF Dataset. Because the directory structure of our benchmarks is ready for training, we can just call `tf.keras.preprocessing.text_dataset_from_directory` function as follows.

In [4]:
BATCH_SIZE = 64
SEQ_PATH = Path.home() / '.genomic_benchmarks' / 'human_nontata_promoters'
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]
NUM_CLASSES = len(CLASSES)

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 27097 files belonging to 2 classes.


In [5]:
if NUM_CLASSES > 2:
    train_dset = train_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

## Text vectorization

To convert the strings to tensors, we internally use TF `TextVectorization` layer and splitting to characters.

In [6]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
VOCAB_SIZE = len(vectorize_layer.get_vocabulary())
vectorize_layer.get_vocabulary()

['', '[UNK]', 'g', 'c', 't', 'a']

In [7]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

To get a baseline (other models can be compared to) we ship a package with [a simple CNN model](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks/blob/main/src/genomic_benchmarks/models/tf.py). We have vectorized the dataset before training the model to speed up the process.

In [8]:
model = get_model(NUM_CLASSES, VOCAB_SIZE)

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0), binary_f1_score])

In [9]:
EPOCHS = 10

history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10
424/424 [==============================] - 23s 13ms/step - loss: 0.4358 - binary_accuracy: 0.8028 - f1_score: 0.7446
Epoch 2/10
424/424 [==============================] - 4s 10ms/step - loss: 0.3771 - binary_accuracy: 0.8323 - f1_score: 0.7944
Epoch 3/10
424/424 [==============================] - 6s 13ms/step - loss: 0.3503 - binary_accuracy: 0.8440 - f1_score: 0.8132
Epoch 4/10
424/424 [==============================] - 5s 12ms/step - loss: 0.3314 - binary_accuracy: 0.8540 - f1_score: 0.8240
Epoch 5/10
424/424 [==============================] - 4s 10ms/step - loss: 0.3134 - binary_accuracy: 0.8631 - f1_score: 0.8404
Epoch 6/10
424/424 [==============================] - 6s 13ms/step - loss: 0.2967 - binary_accuracy: 0.8732 - f1_score: 0.8474
Epoch 7/10
424/424 [==============================] - 4s 10ms/step - loss: 0.2832 - binary_accuracy: 0.8804 - f1_score: 0.8575
Epoch 8/10
424/424 [==============================] - 9s 20ms/step - loss: 0.2719 - binary_accuracy: 0.8861 - 

## Evaluation on the test set

Finally, we can do the same pre-processing for the test set and evaluate the F1 score of our model.

In [10]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

if NUM_CLASSES > 2:
    test_dset = test_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))
test_ds =  test_dset.map(vectorize_text)

Found 9034 files belonging to 2 classes.


In [11]:
model.evaluate(test_ds)

142/142 [==============================] - 2s 12ms/step - loss: 0.3364 - binary_accuracy: 0.8531 - f1_score: 0.8085


[0.3364344537258148, 0.8531104922294617, 0.8084776401519775]